# Imports

In [1]:
import ks_api_client
from ks_api_client import ks_api
import pandas as pd
from datetime import datetime, date
import time
import calendar
import requests
import pandas as pd


# Login Data input

In [2]:
userId=''  
password=''

accessToken=''
consumerKey=''
consumerSecret=''

# Access CODE

In [3]:
accessCode="" # like 1234 ( 4 digit Access code sent by Kotak securities for Login )

# Login 

In [4]:
client = ks_api.KSTradeApi(access_token = accessToken, userid = userId, consumer_key = consumerKey,ip = "192.168.1.9", app_id = "DefaultApplication", \
                        host = "https://ctradeapi.kotaksecurities.com/apim", consumer_secret = consumerSecret)

Login = client.login(password = password)

AC = client.session_2fa(access_code = accessCode)

# Quote Fentching

In [5]:
url = "https://ctradeapi.kotaksecurities.com/apim/scripmaster/1.1/filename"
headers = {'accept':'application/json','consumerKey':f'{consumerKey}','Authorization':f'Bearer {accessToken}'}
res = requests.get(url,headers=headers).json()

# Gobal Order Settings


In [15]:
# order Global SEttings
instrument_name = "BANKNIFTY"
expiry = "22DEC22"
gap = 100
order_type = "MIS"
lots  = 17
timer = 7 # code start before these seconds for 3 OTM Strike Execution
depth = 3 # OTM Depth you want to trade

# Banknifty Token Fetching

In [ ]:
# Banknifty Token Fenching 
cashurl = res['Success']['cash']
cashdata = pd.read_csv(cashurl,sep="|")
cashdata
Banknifty_token = int(cashdata[(cashdata.instrumentName=="NIFTY BANK")]["instrumentToken"])
print(f"BANKNIFTY Token :------- {Banknifty_token}")


# 9:20 Hedge Position

### Hedge strike INFO

In [ ]:
# Hedge Position Info
price = int(float(client.quote(instrument_token = Banknifty_token, quote_type = "LTP")["success"][0]['lastPrice']))
atm = int(round(price/gap,0)*gap)
ce_Hegde = int(input("CE hedge positions : "))
pe_Hedge = int(input("PE hedge positions : "))
print(f"""Banknifty price :------- {price}
Banknifty ATM Stike :--- {atm}
{int((ce_Hegde - atm)/gap)} OTM CE Srike :------- {ce_Hegde}
{int((atm - pe_Hedge)/gap)} OTM PE Strike:------- {pe_Hedge}
""")

# Hedge Strike Token Fetching
fnourl = res['Success']['fno']
fnodata = pd.read_csv(fnourl,sep="|")
CE_token_Hedge = int(fnodata[(fnodata.instrumentType=='OI') & (fnodata.instrumentName==instrument_name)&(fnodata.expiry==expiry)&(fnodata.strike==ce_Hegde)&(fnodata.optionType=="CE")]['instrumentToken'])
PE_token_Hedge = int(fnodata[(fnodata.instrumentType=='OI') & (fnodata.instrumentName==instrument_name)&(fnodata.expiry==expiry)&(fnodata.strike==pe_Hedge)&(fnodata.optionType=="PE")]['instrumentToken'])
CE_Hedge_price = round(float(client.quote(instrument_token = CE_token_Hedge, quote_type = "LTP")["success"][0]['lastPrice']),2)
PE_Hedge_price = round(float(client.quote(instrument_token = PE_token_Hedge, quote_type = "LTP")["success"][0]['lastPrice']),2)
print(f"""CE TOKEN :--{CE_token_Hedge}
PE TOKEN :--{PE_token_Hedge}
Expiry :----{expiry}
{instrument_name} {ce_Hegde} CE Price :  {CE_Hedge_price}
{instrument_name} {pe_Hedge} PE Price :  {PE_Hedge_price}""")


### Hedge Positions Placing


In [10]:
# Hedge Positions Placing 
CE_3OTM = client.place_order(order_type = order_type, instrument_token = CE_token_Hedge, transaction_type = "BUY",\
                   quantity = 25*lots, price = 0, disclosed_quantity = 0, trigger_price = 0,\
                   tag = "string", validity = "GFD", variety = "REGULAR")
PE_3OTM = client.place_order(order_type = order_type, instrument_token = PE_token_Hedge, transaction_type = "BUY",\
                   quantity = 25*lots, price = 0, disclosed_quantity = 0, trigger_price = 0,\
                   tag = "string", validity = "GFD", variety = "REGULAR")


# 9:31 Order Execution

###  Order Execution

In [ ]:
# Execution Time 
otm_3_time = '11:58:00'     # Time at which Strategy has to Deployed on that day

## Second and Minuntes Correction for initiation of Code 
sec = int(otm_3_time[6:8])-timer
min = int(otm_3_time[3:5])
if sec < 0 :
    min = min-1
    sec = 60 - timer
else :
    min = min
    sec = sec

print(f"""Code is Going to Start At :  {otm_3_time[0:2]}:{min}:{sec} 

Trades will Execute At :     {otm_3_time} 

""")
 # Time Trigger for start of Code (since this takes some 3 to 4 seconds to fetch and calculate)
while True :
    if datetime.now().strftime("%H") == otm_3_time[0:2] and int(datetime.now().strftime("%M")) == min and int(datetime.now().strftime("%S")) == sec:
        print("""
        Go trade 16-50 strangle
        """)
        break

print(f"""Strike Calculation Started at  :{time.strftime("%H:%M:%S", time.localtime(time.time()))}
""")
# Strike Calcualtion 
try:
    price = int(float(client.quote(instrument_token = Banknifty_token, quote_type = "LTP")["success"][0]['lastPrice']))
    atm = int(round(price/100,0)*100)
    ce = atm + 100*depth
    pe = atm - 100*depth
except Exception as e : 
    print(f""" error : {e} """)

print(f"""Banknifty price :-------{price}
Banknifty ATM Stike :---{atm}
3 OTM CE Srike :--------{ce}
3 OTM PE Strike:--------{pe}

""")


print(f"""Strike Token Fetced at : {time.strftime("%H:%M:%S", time.localtime(time.time()))}

""")
# Strike Token Fentching
try:
    fnourl = res['Success']['fno']
    fnodata = pd.read_csv(fnourl,sep="|")
    instrument_name = "BANKNIFTY"
    expiry = "22DEC22"
    CE_token = int(fnodata[(fnodata.instrumentType=='OI') & (fnodata.instrumentName==instrument_name)&(fnodata.expiry==expiry)&(fnodata.strike==ce)&(fnodata.optionType=="CE")]['instrumentToken'])
    PE_token = int(fnodata[(fnodata.instrumentType=='OI') & (fnodata.instrumentName==instrument_name)&(fnodata.expiry==expiry)&(fnodata.strike==pe)&(fnodata.optionType=="PE")]['instrumentToken'])
    CE_Price = round(float(client.quote(instrument_token = CE_token, quote_type = "LTP")["success"][0]['lastPrice']),2)
    PE_Price = round(float(client.quote(instrument_token = PE_token, quote_type = "LTP")["success"][0]['lastPrice']),2)
except Exception as e : 
    print(f""" error : {e} """)
print(f"""CE TOKEN :-- {CE_token}
PE TOKEN :-- {PE_token}
Expiry :---- {expiry}
{instrument_name} {ce} CE Price :  {CE_Price}
{instrument_name} {pe} PE Price :  {PE_Price}

""")

print(f"""Order Execution Started at : {time.strftime("%H:%M:%S", time.localtime(time.time()))}

""")
# Order execution at Defined TIME 
try :
    while True :
        if datetime.now().strftime("%H") == otm_3_time[0:2] and datetime.now().strftime("%M") == otm_3_time[3:5] and int(datetime.now().strftime("%S")) == int(otm_3_time[6:8])+5:
            CE_3OTM = client.place_order(order_type = order_type, instrument_token = CE_token, transaction_type = "SELL",\
                        quantity = 25*lots, price = 0, disclosed_quantity = 0, trigger_price = 0,\
                        tag = "string", validity = "GFD", variety = "REGULAR")
            PE_3OTM = client.place_order(order_type = order_type, instrument_token = PE_token, transaction_type = "SELL",\
                        quantity = 25*lots, price = 0, disclosed_quantity = 0, trigger_price = 0,\
                        tag = "string", validity = "GFD", variety = "REGULAR")
            print("""
            
--------------------------Trades Are Executed-----------------------------------
            
            """)
            break
except Exception as e : 
    print(f""" error : {e} """)

print(f"""Order Execution Completed at : {time.strftime("%H:%M:%S", time.localtime(time.time()))}""")


### StopLoss 

In [ ]:
# fetch position data from Trading Account
c = client.positions(position_type = "TODAYS")['Success']
# StopLoss Placment
try:
    for i in c :
        for j in i :
            if j == 'strikePrice' :
                if int(i[j]) == ce:
                    slce = round(round((float(i['sellTrdAvg'])*1.16+0.1)/0.05,0)*0.05,2)
                    trigce = round(slce-0.1,2)    
                    print(f"""{ce} CE Trigger  : {trigce}
{ce} CE StopLoss : {slce}
""")
                    CE_3OTM = client.place_order(order_type = order_type, instrument_token = CE_token, transaction_type = "BUY",\
                            quantity = 25*lots, price = slce, disclosed_quantity = 0, trigger_price = trigce,\
                            tag = "string", validity = "GFD", variety = "REGULAR")
                    
                elif int(i[j]) == pe:
                    slpe = round(round((float(i['sellTrdAvg'])*1.16+0.1)/0.05,0)*0.05,2)
                    trigpe = round(slpe-0.1,2)   
                    print(f"""{pe} CE Trigger  : {trigpe}
{pe} CE StopLoss : {slpe}""")
                    CE_3OTM = client.place_order(order_type = order_type, instrument_token = PE_token, transaction_type = "BUY",\
                            quantity = 25*lots, price = slpe, disclosed_quantity = 0, trigger_price = trigpe,\
                            tag = "string", validity = "GFD", variety = "REGULAR")

    print("------------------------------------StopLoss Exected----------------------------------------------------------")
except Exception as e: 
    print(f""" error : {e} """)

# StopLoss Targets SUMMARY
print("""--------------------: StopLoss Targets SUMMARY :--------------------""")
try:
    for i in c :
        for j in i :
            if j == 'strikePrice' :
                if int(i[j]) == ce:
                    slce = round(round((i['sellTrdAvg']*1.16)/0.05,0)*0.05,2)
                    trigce = round(slce-0.1,2)
                    print(f"""Strike : {i['instrumentName']}
    Sell Price : {i['sellTrdAvg']}
    Trigger SL..! : {round(i['sellTrdAvg']*1.16,1)}
    Stoploss....! : {round(i['sellTrdAvg']*1.16+0.15,1)}
    Target 35 : {i['sellTrdAvg']*0.65}
    Target 40 : {i['sellTrdAvg']*0.6}
    Target 50 : {i['sellTrdAvg']*0.495}
                    """)
                elif int(i[j]) == pe:  
                        print(f"""Strike : {i['instrumentName']}
    Sell Price : {i['sellTrdAvg']}
    Trigger SL..! : {round(i['sellTrdAvg']*1.16,1)}
    Stoploss....! : {round(i['sellTrdAvg']*1.16+0.15,1)}
    Target 35 : {i['sellTrdAvg']*0.65}
    Target 40 : {i['sellTrdAvg']*0.6}
    Target 50 : {i['sellTrdAvg']*0.495}
                    """)
except Exception as e:
    print(f""" error : {e} """)